इस लेख में शामिल जप्टर नोटबुक [अध्याय 4 कोड आधार](https://github.com/datawhalechina/learn-nlp-with-transformers/tree/main/docs/%E7%AF%87%E7%AB) में है %A04 -%E4%BD%BF%E7%94%A8ट्रांसफॉर्मर%E8%A7%A3%E5%86%B3NLP%E4%BB%BB%E5%8A%A1)।

आप इस ट्यूटोरियल को खोलने और प्रासंगिक डेटा सेट और मॉडल डाउनलोड करने के लिए सीधे Google कोलाब नोटबुक का भी उपयोग कर सकते हैं।
यदि आप इस नोटबुक को Google के कोलाब में खोल रहे हैं, तो आपको ट्रांसफॉर्मर और 🤗डेटासेट लाइब्रेरी इंस्टॉल करने की आवश्यकता हो सकती है। इसे स्थापित करने के लिए निम्नलिखित कमांड को अनकम्मेंट करें।

In [ ]:
!pip install transformers datasets

यदि आप इस नोटबुक को स्थानीय रूप से खोल रहे हैं, तो सुनिश्चित करें कि आपने उपरोक्त निर्भरता पैकेज स्थापित किए हैं।
आप यहां इस नोटबुक का मल्टी-जीपीयू वितरित प्रशिक्षण संस्करण भी पा सकते हैं।

# पाठ वर्गीकरण के लिए पूर्व-प्रशिक्षित मॉडल को ठीक करें

हम दिखाएंगे कि [GLUE बेंचमार्क] (https://ग्लूबेंचमार्क.com/) से पाठ वर्गीकरण कार्यों को हल करने के लिए [🤗 ट्रांसफॉर्मर्स] (https://github.com/huggingface/transformers) कोड बेस से मॉडल का उपयोग कैसे करें।

![पाठ वर्गीकरण कार्य पर विजेट अनुमान](https://github.com/huggingface/notebooks/blob/master/examples/images/text_classification.png?raw=1)

GLUE सूची में 9 वाक्य-स्तरीय वर्गीकरण कार्य शामिल हैं, अर्थात्:
- [CoLA](https://nyu-mll.github.io/CoLA/) (भाषाई स्वीकार्यता का कोष) पहचानें कि कोई वाक्य व्याकरणिक रूप से सही है या नहीं।
- [एमएनएलआई](https://arxiv.org/abs/1704.05426) (बहु-शैली प्राकृतिक भाषा अनुमान) एक परिकल्पना दी गई है, दूसरे वाक्य और परिकल्पना के बीच संबंध निर्धारित करें: विवरण, विरोधाभास या असंबंधित।
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft रिसर्च पैराफ्रेज़ कॉर्पस) निर्धारित करें कि क्या दो वाक्य एक दूसरे के पैराफ्रेज़ हैं।
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (प्रश्न-उत्तर देने वाला प्राकृतिक भाषा अनुमान) निर्धारित करें कि क्या दूसरे वाक्य में पहले वाक्य का उत्तर है।
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora प्रश्न जोड़े2) निर्धारित करें कि क्या दो प्रश्नों का शब्दार्थ समान है।
- [आरटीई](https://aclweb.org/aclwiki/Recognising_Textual_Entailment) (पहचाननाटेक्स्टुअल एंटेलमेंट) यह निर्धारित करता है कि कोई वाक्य परिकल्पना में शामिल है या नहीं।
- [एसएसटी-2](https://nlp.stanford.edu/sentiment/index.html) (स्टैनफोर्ड सेंटिमेंट ट्रीबैंक) एक वाक्य की सकारात्मक या नकारात्मक भावना निर्धारित करें।
- [एसटीएस-बी](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (सिमेंटिक टेक्स्टुअल समानता बेंचमार्क) दो वाक्यों की समानता निर्धारित करें (स्कोर 1-5 अंक है)।
- [डब्ल्यूएनएलआई](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (विनोग्राड प्राकृतिक भाषा अनुमान) निर्धारित करें कि क्या अनाम सर्वनाम वाला वाक्य और इस सर्वनाम को प्रतिस्थापित करने वाला वाक्य शामिल है या नहीं।

पिछले के लिएएक कार्य के रूप में, हम दिखाएंगे कि डेटासेट को लोड करने के लिए सरल डेटासेट लाइब्रेरी का उपयोग कैसे करें, और साथ ही पूर्व-प्रशिक्षित मॉडल को ठीक करने के लिए ट्रांसफार्मर में `ट्रेनर` इंटरफ़ेस का उपयोग करें।

In [10]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

यह नोटबुक किसी भी पाठ वर्गीकरण कार्य को हल करने के लिए सैद्धांतिक रूप से विभिन्न ट्रांसफार्मर मॉडल ([मॉडल पैनल] (https://huggingface.co/models)) का उपयोग कर सकता है।

यदि आप जिन कार्यों से निपट रहे हैं वे अलग-अलग हैं, तो आप केवल मामूली बदलावों के साथ इस नोटबुक का उपयोग करने में सक्षम होंगे। साथ ही, आपको मेमोरी ओवरफ़्लो से बचने के लिए अपनी GPU मेमोरी के अनुसार फ़ाइन-ट्यूनिंग प्रशिक्षण के लिए आवश्यक btach आकार को समायोजित करना चाहिए।

In [11]:
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## डेटा डाउनलोड करें

हम डेटा और संबंधित मूल्यांकन विधियों को लोड करने के लिए [🤗 डेटासेट](https://github.com/huggingface/datasets) लाइब्रेरी का उपयोग करेंगे। डेटा लोडिंग और मूल्यांकन मोड लोडिंग के लिए केवल `load_dataset` और `load_metric` के सरल उपयोग की आवश्यकता होती है।

In [12]:
from datasets import load_dataset, load_metric

`एमएनएलआई-एमएम` को छोड़कर, अन्य कार्यों को सीधे कार्य नाम के माध्यम से लोड किया जा सकता है। लोड होने के बाद डेटा स्वचालित रूप से कैश हो जाता है।

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

`डेटासेट` ऑब्जेक्ट स्वयं एक [`डेटासेटडिक्ट`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict) डेटा संरचना है, प्रशिक्षण सेट, सत्यापन सेट और परीक्षण सेट के लिए, बस उपयोग करें संबंधित डेटा प्राप्त करने के लिए संबंधित कुंजी (ट्रेन, सत्यापन, परीक्षण)।

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

डेटा विभाजन कुंजी (ट्रेन, सत्यापन, या परीक्षण) और सबस्क्रिप्ट दिए जाने पर, आप डेटा देख सकते हैं।

In [15]:
dataset["train"][0]

{'idx': 0,
 'label': 1,
 'sentence': "Our friends won't buy this analysis, let alone the next one we propose."}

यह समझने के लिए कि डेटा कैसा दिखता है, निम्न फ़ंक्शन प्रदर्शित करने के लिए डेटा सेट से यादृच्छिक रूप से कुछ उदाहरणों का चयन करेगा।

In [16]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [17]:
show_random_elements(dataset["train"])

,sentence,label,idx
0,"The more I talk to Joe, the less about linguistics I am inclined to think Sally has taught him to appreciate.",acceptable,196
1,Have in our class the kids arrived safely?,unacceptable,3748
2,I gave Mary a book.,acceptable,5302
3,"Every student, who attended the party, had a good time.",unacceptable,4944
4,Bill pounded the metal fiat.,acceptable,2178
5,It bit me on the leg.,acceptable,5908
6,The boys were made a good mother by Aunt Mary.,unacceptable,736
7,More of a man is here.,unacceptable,5403
8,My mother baked me a birthday cake.,acceptable,3761
9,Gregory appears to have wanted to be loyal to the company.,acceptable,4334


मूल्यांकन मेट्रिक [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric) का एक उदाहरण है:

In [18]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

मीट्रिक की 'गणना' पद्धति को सीधे कॉल करें और मीट्रिक का मान प्राप्त करने के लिए 'लेबल' और 'भविष्यवाणियां' पास करें:

In [19]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'matthews_correlation': 0.1513518081969605}

प्रत्येक पाठ वर्गीकरण कार्य के अनुरूप मेट्रिक इस प्रकार भिन्न है:

- CoLA के लिए: [मैथ्यूज़ सहसंबंध गुणांक](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- एमएनएलआई के लिए (मिलान या बेमेल): सटीकता
- एमआरपीसी के लिए: सटीकता और [एफ1 स्कोर](https://en.wikipedia.org/wiki/F1_score)
- क्यूएनएलआई के लिए: सटीकता
- QQP के लिए: सटीकता और [F1 स्कोर](https://en.wikipedia.org/wiki/F1_score)
- आरटीई के लिए: सटीकता
- एसएसटी-2 के लिए: सटीकता
- एसटीएस-बी के लिए: [पियर्सन सहसंबंध गुणांक](https://en.wikipedia.org/wiki/Pearson_correlation_coeकुशल) और [स्पीयरमैन_रैंक_सहसंबंध_गुणांक](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- डब्ल्यूएनएलआई के लिए: सटीकता

इसलिए मीट्रिक को कार्य के साथ संरेखित करना सुनिश्चित करें

## डेटा प्रीप्रोसेसिंग

मॉडल में डेटा फीड करने से पहले, हमें डेटा को प्रीप्रोसेस करना होगा। प्रीप्रोसेसिंग टूल को 'टोकनाइज़र' कहा जाता है। `टोकनाइज़र` पहले इनपुट को टोकनाइज़ करता है, फिर टोकन को प्री-मॉडल में आवश्यक संबंधित टोकन आईडी में परिवर्तित करता है, और फिर इसे मॉडल द्वारा आवश्यक इनपुट प्रारूप में परिवर्तित करता है।

डेटा प्रीप्रोसेसिंग के उद्देश्य को प्राप्त करने के लिए, हम अपने टोकननाइज़र को इंस्टेंट करने के लिए `AutoTokenizer.from_pretrained` विधि का उपयोग करते हैं, जो सुनिश्चित करता है:

- हमें एक टोकननाइज़र मिलता है जो पूर्व-प्रशिक्षित मॉडल वन-टू-वन से मेल खाता है।
- निर्दिष्ट मॉडल चेकपॉइंट के अनुरूप टोकननाइज़र का उपयोग करते समय, हम मॉडल द्वारा आवश्यक शब्दावली लाइब्रेरी भी डाउनलोड करते हैं, सटीक होने के लिए, टोकन शब्दावली।

डाउनलोड किए गए टोकन शब्दावली को कैश किया जाएगा ताकि दोबारा उपयोग करने पर इसे दोबारा डाउनलोड न किया जा सके।

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

ध्यान दें: `use_fast=True` के लिए आवश्यक है कि टोकननाइज़र ट्रांसफार्मर.PreTrainedTokenizerFast प्रकार का होना चाहिए, क्योंकि हमें प्रीप्रोसेसिंग के दौरान फास्ट टोकननाइज़र की कुछ विशेष सुविधाओं (जैसे मल्टी-थ्रेडेड फास्ट टोकननाइज़र) का उपयोग करने की आवश्यकता होती है। यदि संबंधित मॉडल में तेज़ टोकननाइज़र नहीं है, तो बस इस विकल्प को हटा दें।

मॉडलों के अनुरूप लगभग सभी टोकननाइजरों में संगत तेज टोकननाइजर होते हैं। हम [मॉडल टोकनाइज़र कॉरेस्पोंडेंस टेबल] (https://huggingface.co/transformers/index.html#bigtable) में सभी पूर्व-प्रशिक्षित मॉडलों के अनुरूप टोकननाइज़र की विशेषताओं को देख सकते हैं।

टोकननाइज़र या तो एक टेक्स्ट या टेक्स्ट की एक जोड़ी को प्रीप्रोसेस कर सकता है। टोकननाइज़र प्रीप्रोसेसिंग के बाद प्राप्त डेटा प्री-ट्रेनिंग मॉडल के इनपुट प्रारूप से मिलता है।

In [21]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

हमारे द्वारा चुने गए पूर्व-प्रशिक्षित मॉडल के आधार पर, हम टोकनाइज़र से अलग-अलग रिटर्न देखेंगे। टोकनाइज़र और पूर्व-प्रशिक्षित मॉडल के बीच एक-से-एक पत्राचार है। अधिक जानकारी यहां पाई जा सकती है। /huggingface.co/transformers/ preprocessing.html) सीखने के लिए।

अपने डेटा को प्रीप्रोसेस करने के लिए, हमें विभिन्न डेटा और संबंधित डेटा प्रारूपों को जानना होगा, इसलिए हम निम्नलिखित निर्देश को परिभाषित करते हैं।

In [22]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

डेटा प्रारूप जांचें:

In [23]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Our friends won't buy this analysis, let alone the next one we propose.


फिर प्रीप्रोसेसिंग कोड को एक फ़ंक्शन में डालें:

In [24]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

प्रीप्रोसेसिंग फ़ंक्शन एकल नमूने या एकाधिक नमूनों को संसाधित कर सकते हैं। यदि इनपुट एकाधिक नमूने है, तो एक सूची लौटा दी जाती है:

In [25]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 1998, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 2030, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 1996, 2062, 2057, 2817, 16025, 1010, 1996, 13675, 16103, 2121, 2027, 2131, 1012, 102], [101, 2154, 2011, 2154, 1996, 8866, 2024, 2893, 14163, 8024, 3771, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

इसके बाद, डेटासेट में सभी नमूने पूर्व-संसाधित होते हैं। प्रसंस्करण की विधि मैप फ़ंक्शन का उपयोग करना और सभी नमूनों पर प्रीप्रोसेसिंग फ़ंक्शन तैयार_ट्रेन_फीचर्स को लागू करना है।

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

इससे भी बेहतर, अगली बार संसाधित होने पर पुनर्गणना से बचने के लिए लौटाए गए परिणाम स्वचालित रूप से कैश किए जाएंगे (लेकिन कृपया यह भी ध्यान दें कि यदि इनपुट बदल दिया जाता है, तो यह कैश से प्रभावित हो सकता है!)। डेटासेट लाइब्रेरी फ़ंक्शन यह निर्धारित करने के लिए इनपुट पैरामीटर का पता लगाएगा कि क्या परिवर्तन हैं। यदि कोई परिवर्तन नहीं हैं, तो कैश्ड डेटा का उपयोग किया जाएगा। यदि परिवर्तन हैं, तो डेटा को पुन: संसाधित किया जाएगा। लेकिन यदि इनपुट पैरामीटर अपरिवर्तित रहते हैं और आप इनपुट बदलना चाहते हैं, तो कैश को साफ़ करना और समायोजित करना सबसे अच्छा है। साफ़ करने का तरीका `load_from_cache_file=False` पैरामीटर का उपयोग करना है। इसके अलावा, ऊपर इस्तेमाल किया गया `बैच = ट्रू` पैरामीटर टोकननाइज़र की एक विशेषता है, जिसके बारे में माना जाता है कि यह एक ही समय में समानांतर में इनपुट को संसाधित करने के लिए कई थ्रेड्स का उपयोग करता है।

## पूर्व-प्रशिक्षित मॉडल को फाइन-ट्यून करें

अब जब डेटा तैयार हो गया है, तो हमें अपने पूर्व-प्रशिक्षित मॉडल को डाउनलोड और लोड करना होगा, और फिर पूर्व-प्रशिक्षित मॉडल को फाइन-ट्यून करना होगा। चूँकि हम seq2seq कार्य कर रहे हैं, हमें एक मॉडल वर्ग की आवश्यकता है जो इस कार्य को हल कर सके। हम वर्ग `AutoModelForSequenceClassification` का उपयोग करते हैं। टोकननाइज़र के समान, `from_pretrained` विधि भी हमें मॉडल को डाउनलोड और लोड करने में मदद कर सकती है, यह मॉडल को कैश भी करेगी ताकि मॉडल बार-बार डाउनलोड न हो।

यह ध्यान दिया जाना चाहिए कि एसटीएस-बी एक प्रतिगमन समस्या है और एमएनएलआई एक 3-वर्गीकरण समस्या है:

In [27]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

चूँकि हमारा फाइन-ट्यूनिंग कार्य एक पाठ वर्गीकरण कार्य है, और हम एक पूर्व-प्रशिक्षित भाषा मॉडल लोड कर रहे हैं, मॉडल लोड करते समय हमें कुछ बेमेल तंत्रिका नेटवर्क मापदंडों को हटाने के लिए प्रेरित किया जाएगा (उदाहरण के लिए: पूर्व का तंत्रिका नेटवर्क प्रमुख) -प्रशिक्षित भाषा मॉडल को फेंक दिया गया था, जबकि पाठ वर्गीकरण के लिए तंत्रिका नेटवर्क प्रमुख को यादृच्छिक रूप से प्रारंभ किया गया था)।

`ट्रेनर` प्रशिक्षण उपकरण प्राप्त करने के लिए, हमें 3 तत्वों की भी आवश्यकता है, जिनमें से सबसे महत्वपूर्ण है प्रशिक्षण सेटिंग्स/पैरामीटर [`ट्रेनिंगआर्ग्यूमेंट्स`](https://huggingface.co/transformers/main_classes/trainer.html# ट्रांसफार्मर। प्रशिक्षण तर्क)। इस प्रशिक्षण सेटिंग में वे सभी गुण शामिल हैं जो प्रशिक्षण प्रक्रिया को परिभाषित करते हैं।

In [28]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

उपरोक्त मूल्यांकन_रणनीति = "युग" पैरामीटर प्रशिक्षण कोड बताता है: हम प्रत्येक युग के लिए एक सत्यापन मूल्यांकन करेंगे।

उपरोक्त बैच_आकार इस नोटबुक से पहले परिभाषित किया गया था।

अंत में, चूंकि विभिन्न कार्यों के लिए अलग-अलग मूल्यांकन संकेतकों की आवश्यकता होती है, हम कार्य नाम के आधार पर मूल्यांकन पद्धति प्राप्त करने के लिए एक फ़ंक्शन को परिभाषित करते हैं:

In [29]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

सभी को `ट्रेनर` को पास करें:

In [30]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

प्रशिक्षण शुरू करो:

In [31]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence.
***** Running training *****
  Num examples = 8551
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2675


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.525400,0.520955,0.409248
2,0.351600,0.570341,0.477499
3,0.236100,0.622785,0.499872
4,0.166300,0.806475,0.491623
5,0.125700,0.882225,0.513900


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16
Saving model checkpoint to test-glue/checkpoint-535
Configuration saved in test-glue/checkpoint-535/config.json
Model weights saved in test-glue/checkpoint-535/pytorch_model.bin
tokenizer config file saved in test-glue/checkpoint-535/tokenizer_config.json
Special tokens file saved in test-glue/checkpoint-535/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16
Saving model checkpoint to test-glue/checkpoint-1070
Configuration saved in test-glue/checkpoint-1070/config.json
Model weights saved in test-glue/checkpoint-1070/pytorch_model.bin
tokeniz

TrainOutput(global_step=2675, training_loss=0.27181456521292713, metrics={'train_runtime': 169.649, 'train_samples_per_second': 252.02, 'train_steps_per_second': 15.768, 'total_flos': 229537542078168.0, 'train_loss': 0.27181456521292713, 'epoch': 5.0})

प्रशिक्षण पूरा होने के बाद मूल्यांकन:

In [32]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 16


{'epoch': 5.0,
 'eval_loss': 0.8822253346443176,
 'eval_matthews_correlation': 0.5138995234247261,
 'eval_runtime': 0.9319,
 'eval_samples_per_second': 1119.255,
 'eval_steps_per_second': 70.825}

यह देखने के लिए कि आपके मॉडल का प्रदर्शन कैसा रहा, आप इसकी तुलना [GLUE बेंचमार्क लीडरबोर्ड](https://gloebenchmark.com/leaderboard) से कर सकते हैं।

## हाइपरपैरामीटर खोज

`ट्रेनर` [ऑप्टुना] (https://optona.org/) या [रे ट्यून] (https://docs.ray.io/en/latest/tune/) कोड लाइब्रेरी का उपयोग करके हाइपरपैरामीटर खोज का भी समर्थन करता है।

निर्भरताएँ स्थापित करने के लिए निम्नलिखित दो पंक्तियों को अनकम्मेंट करें:

In [ ]:
! pip install optuna
! pip install ray[tune]

सुपर मापदंडों की खोज करते समय, `ट्रेनर` कई प्रशिक्षित मॉडल लौटाएगा, इसलिए एक परिभाषित मॉडल को पारित करने की आवश्यकता है ताकि `ट्रेनर` लगातार पारित मॉडल को पुन: प्रारंभ कर सके:

In [34]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

पहले `ट्रेनर` को कॉल करने के समान:

In [35]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.9.1",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9a

कॉल विधि `हाइपरपैरामीटर_सर्च`। ध्यान दें कि इस प्रक्रिया में लंबा समय लग सकता है। हम पहले हाइपरपैरामीटर खोज करने के लिए डेटा सेट के हिस्से का उपयोग कर सकते हैं और फिर पूर्ण प्रशिक्षण कर सकते हैं।
उदाहरण के लिए, खोजने के लिए डेटा का 1/10 भाग का उपयोग करें:

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

`hyperparameter_search` सर्वोत्तम प्रदर्शन करने वाले मॉडल से संबंधित पैरामीटर लौटाएगा:

In [ ]:
best_run

`ट्रेनर` को सर्वोत्तम खोजे गए मापदंडों पर सेट करें और प्रशिक्षित करें:

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

अंत में, यह देखना न भूलें कि मॉडल कैसे अपलोड करें, मॉडल को](https://huggingface.co/transformers/model_sharing.html) से [🤗 मॉडल हब](https://huggingface.co/models) पर अपलोड करें। . फिर आप सीधे मॉडल नाम का उपयोग करके अपने द्वारा अपलोड किए गए मॉडल का उपयोग कर सकते हैं जैसे आपने इस नोटबुक की शुरुआत में किया था।